# Prerequesites

Install kaggle

In [1]:
!pip install kaggle
!pip install tensorflow-plot

    100% |████████████████████████████████| 61kB 3.9MB/s 
    100% |████████████████████████████████| 235kB 7.3MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify
  Running setup.py bdist_wheel for tensorflow-plot ... - done
  Stored in directory: /root/.cache/pip/wheels/aa/0d/52/6e050bed5bfe6e39964a4a6e3ba8a0c0e64d37f42ac7af1a75
Successfully built tensorflow-plot


You will be prompted to Authorize Google Cloud SDK to access the content of your Google Drive

In [2]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


Download the HAM10000 dataset using the kaggle API

In [3]:
!kaggle datasets download kmader/skin-cancer-mnist-ham10000

100% 2.60G/2.62G [00:27<00:00, 95.4MB/s]
100% 2.62G/2.62G [00:28<00:00, 100MB/s] 


Time to unzip the dataset und to remove the  zip-files after unzipping.

In [4]:
!mkdir -p /content/dataset_skin
!unzip -n skin-cancer-mnist-ham10000.zip -d /content/dataset_skin
!unzip -n -q /content/dataset_skin/HAM10000_images_part_1.zip -d /content/dataset_skin/images
!unzip -n -q /content/dataset_skin/HAM10000_images_part_2.zip -d /content/dataset_skin/images
!rm /content/dataset_skin/HAM10000_images_part_*.zip
!rm skin-cancer-mnist-ham10000.zip

Archive:  skin-cancer-mnist-ham10000.zip
  inflating: /content/dataset_skin/hmnist_28_28_RGB.csv  
  inflating: /content/dataset_skin/HAM10000_metadata.csv  
  inflating: /content/dataset_skin/HAM10000_images_part_1.zip  
  inflating: /content/dataset_skin/hmnist_28_28_L.csv  
  inflating: /content/dataset_skin/hmnist_8_8_L.csv  
  inflating: /content/dataset_skin/HAM10000_images_part_2.zip  
  inflating: /content/dataset_skin/hmnist_8_8_RGB.csv  


Now we make the necessary steps to mount the Google Drive directory in our Python environment by using the `google-drive-ocamlfuse` API.

Once again, you will be asked again for permission to allow Google Cloud SDK to access your Google Drive content.

In [5]:
# installing stuff
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Authenticate
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

Now we proceed by cloning my classifier repo from github.com

In [0]:
#!git clone https://github.com/jzuern/traffic-light-recognition

# Monitor training using TensorBoard

Many thanks to [this stackoverflow answer](https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab#48468512)


First, let's execute TensorBoard in the environment specifying host and port

In [7]:
# First, download and install ngrok:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2018-10-29 21:13:21--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.0.94.50, 52.203.53.176, 52.201.75.180, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.0.94.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  9.62MB/s    in 0.5s    

2018-10-29 21:13:21 (9.62 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [32]:
# #kill all running ngrok instances
!pkill -f ngrok

# Execute tensorboard
LOG_DIR = '/content/drive/classifier/checkpoints/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# execute ngrok
get_ipython().system_raw('./ngrok http 6006 &')

# Do the tunneling
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://592d5fa8.ngrok.io


In [38]:
!rm /content/dataset_skin/*.tfrecords
!rm -rf /content/drive/classifier/checkpoints/*

rm: cannot remove '/content/drive/classifier/checkpoints/eval': Directory not empty


# **Start the classifier!**

Now we mount our Google Drive directory to `/content/drive`

**Important note**: The directory has to be remounted every time Google Drive content changes! --> Every time your code changes

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive

In [0]:
!python3 /content/drive/classifier/main.py

Serializing sample 0/10016
Serializing sample 100/10016
Serializing sample 200/10016
Serializing sample 300/10016
Serializing sample 400/10016
Serializing sample 500/10016
Serializing sample 600/10016
Serializing sample 700/10016
Serializing sample 800/10016
Serializing sample 900/10016
Serializing sample 1000/10016
Serializing sample 1100/10016
Serializing sample 1200/10016
Serializing sample 1300/10016
Serializing sample 1400/10016
Serializing sample 1500/10016
Serializing sample 1600/10016
Serializing sample 1700/10016
Serializing sample 1800/10016
Serializing sample 1900/10016
Serializing sample 2000/10016
Serializing sample 2100/10016
Serializing sample 2200/10016
Serializing sample 2300/10016
Serializing sample 2400/10016
Serializing sample 2500/10016
Serializing sample 2600/10016
Serializing sample 2700/10016
Serializing sample 2800/10016
Serializing sample 2900/10016
Serializing sample 3000/10016
Serializing sample 3100/10016
Serializing sample 3200/10016
Serializing sample 330

# Ablation study

1.   Number of residual blocks
2.   Type of activation function

